<a href="https://colab.research.google.com/github/tonchan1216/bitbot/blob/master/apply_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import numpy as np
import random
from google.colab import drive
import copy
import math

from datetime import datetime
import pickle

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, ReLU
from tensorflow.keras.optimizers import RMSprop
from sklearn.preprocessing import StandardScaler

In [19]:
class Brain:
    def __init__(self):
        optimizer = RMSprop()

        model = Sequential()
        model.add(Dense(3, input_shape=(3,)))
        model.add(ReLU()) 
        model.add(Dense(3))
        model.add(ReLU())
        model.add(Dense(3))
        model.compile(loss="mse", optimizer=optimizer)
        self.model = model

    def predict(self, state):
        return self.model.predict(state)

    def load(self, name):
        self.model.load_weights(name)

In [20]:
class Agent(Brain):
    def __init__(self):
        super().__init__()
        self.epsilon = 0.01

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(3)
        act_values = self.predict(state)
        return np.argmax(act_values[0])

In [21]:
def get_now_state():
    state = np.empty(3)
    state[0] = 0 ## 保有するポジション
    state[1] = 0 ## 現在のレート
    state[2] = 0 ## 保有する日本円現金
    return state

def trade(action):
    # 買い
    if action_space[0] == action and hold_a_position == 0:
          before_buy_cash = copy.copy(cash_in_hand)
          if cash_in_hand < now_price * min_trade_btc + commission * now_price:
            return

          hold_a_position = math.floor(((cash_in_hand/now_price) - commission) * 10000) / 10000 
          buy_price += now_price * hold_a_position
          cash_in_hand -= now_price * hold_a_position + commission * now_price
    # 売り
    if action_space[2] == action and hold_a_position != 0:
          sell_price += now_price * hold_a_position
          cash_in_hand += now_price * hold_a_position - commission * now_price
          hold_a_position = 0
          trade_time += 1
          if cash_in_hand > before_buy_cash:
              trade_win += 1

In [22]:
name = 'qlearning'
drive.mount('/content/drive/')
mdl_dir = '/content/drive/My Drive/Colab Notebooks/models'

agent = Agent()
with open('{}/{}.pkl'.format(mdl_dir, name), 'rb') as f:
    scaler = pickle.load(f)
agent.load('{}/{}.h5'.format(mdl_dir, name))
state = scaler.transform([get_now_state()])
action = agent.act(state)
# trade(action)
print(action)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
0
